In [182]:
import datetime
import pandas as pd
import numpy as np
from sklearn import linear_model
from google.cloud import bigquery


In [183]:
# Function that calls SQL code that fetches the IATA tables
def load_table(query):
    client=bigquery.Client()
    query_job=client.query(query)
    print("Reading table" )
    results = query_job.result().to_dataframe()
    print('Compiled the dataframe')
    df_lines = len(results.index)
    print('Processed {0} lines'.format(df_lines))
    return results

In [184]:
alert_table_DF = pd.DataFrame()

In [185]:
%%bigquery D_flight_capacity_DF
SELECT
  FLIGHT_DATE,
  COUNT(FLIGHT_DATE) AS number_fligt,
  SUM(overbooked_flight) AS overbooked,
  SUM(low_booking_flight) AS low_booking
FROM `compliance_dashboard_IN1194.D_flight_capacity`
WHERE FLIGHT_DATE > "2020-03-08"
  AND FLIGHT_DATE < "2020-07-01"
  GROUP BY FLIGHT_DATE

In [186]:
D_flight_capacity_DF["over_ratio"] = D_flight_capacity_DF["overbooked"]/D_flight_capacity_DF["number_fligt"]
D_flight_capacity_DF["under_ratio"] = D_flight_capacity_DF["low_booking"]/D_flight_capacity_DF["number_fligt"]
D_flight_capacity_DF["FLIGHT_DATE"] = pd.to_datetime(D_flight_capacity_DF['FLIGHT_DATE'])

In [187]:
D_flight_capacity_DF_overbooked = D_flight_capacity_DF[D_flight_capacity_DF["over_ratio"] > 0.02]

In [188]:
alert_table_DF["Date"] = D_flight_capacity_DF_overbooked.FLIGHT_DATE
alert_table_DF["Date_type"] = "Flight Date"
alert_table_DF["Alert"] = "too many overbooked flights for the day"
alert_table_DF["value"] = D_flight_capacity_DF_overbooked.over_ratio
alert_table_DF

,Date,Date_type,Alert,value


In [189]:
D_flight_capacity_DF_underbooked = D_flight_capacity_DF[D_flight_capacity_DF["under_ratio"] > 0.65].sort_values(by = "FLIGHT_DATE")

In [190]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = D_flight_capacity_DF_underbooked.FLIGHT_DATE
alert_table_DF_tmp["Date_type"] = "Flight Date"
alert_table_DF_tmp["Alert"] = "too many underbooked flights for the day"
alert_table_DF_tmp["value"] = D_flight_capacity_DF_underbooked.over_ratio
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)
alert_table_DF.head()

,Date,Date_type,Alert,value
42,2020-03-17,Flight Date,too many underbooked flights for the day,0.005124
37,2020-03-18,Flight Date,too many underbooked flights for the day,0.005869
18,2020-03-21,Flight Date,too many underbooked flights for the day,0.009055
95,2020-03-22,Flight Date,too many underbooked flights for the day,0.007180
24,2020-03-23,Flight Date,too many underbooked flights for the day,0.004306


In [191]:
%%bigquery NewAirport_DF
SELECT * 
FROM `compliance_dashboard_IN1194.D_First_time_outbound_airport`
WHERE first_time_seen > "2020-01-01"

In [192]:
NewAirport_DF["first_time_seen"] = pd.to_datetime(NewAirport_DF['first_time_seen'])
NewAirport_DF

,flyfrom_Leg,first_time_seen,number_tickets
0,MWQ,2020-01-02,41
1,TSM,2020-02-03,7
2,JGB,2020-03-05,29
3,GYB,2020-05-18,87
4,LHU,2020-01-02,1
5,OLM,2020-01-02,1
6,BKN,2020-01-24,1
7,HUF,2020-01-28,1
8,RAL,2020-03-03,1
9,CDH,2020-03-03,1


In [193]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = NewAirport_DF.first_time_seen
alert_table_DF_tmp["Date_type"] = "Issue date"
alert_table_DF_tmp["Alert"] = "Outbound airport seen for the first time"
alert_table_DF_tmp["value"] = NewAirport_DF["flyfrom_Leg"]  
alert_table_DF_tmp
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)

In [194]:
%%bigquery NewAirport_DF
SELECT * 
FROM `compliance_dashboard_IN1194.D_First_time_inbound_airport` 
WHERE first_time_seen > "2020-03-08"

In [195]:
NewAirport_DF["first_time_seen"] = pd.to_datetime(NewAirport_DF['first_time_seen'])
NewAirport_DF

,flyto_leg,first_time_seen,number_tickets
0,GYB,2020-05-18,100
1,KXN,2020-03-28,1
2,LSF,2020-04-15,5


In [196]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = NewAirport_DF.first_time_seen
alert_table_DF_tmp["Date_type"] = "Issue date"
alert_table_DF_tmp["Alert"] = "Inboud airport seen for the first time"
alert_table_DF_tmp["value"] = NewAirport_DF["flyto_leg"]  
alert_table_DF_tmp
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)

In [197]:
%%bigquery NewAirport_DF
SELECT * 
FROM `compliance_dashboard_IN1194.D_First_time_inbound_airport` 
WHERE first_time_seen > "2020-03-08"

In [198]:

NewAirport_DF["first_time_seen"] = pd.to_datetime(NewAirport_DF['first_time_seen'])
NewAirport_DF

,flyto_leg,first_time_seen,number_tickets
0,GYB,2020-05-18,100
1,KXN,2020-03-28,1
2,LSF,2020-04-15,5


In [199]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = NewAirport_DF.first_time_seen
alert_table_DF_tmp["Date_type"] = "Issue date"
alert_table_DF_tmp["Alert"] = "Inboud airport seen for the first time"
alert_table_DF_tmp["value"] = NewAirport_DF["flyto_leg"]  
alert_table_DF_tmp
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)

In [200]:
%%bigquery NewRoute_DF
SELECT * 
FROM `compliance_dashboard_IN1194.D_First_time_itinerary` 
WHERE first_time_seen > "2020-03-08" AND  number_tickets >250

In [201]:
NewRoute_DF["first_time_seen"] = pd.to_datetime(NewRoute_DF['first_time_seen'])
NewRoute_DF.head()

,flyfrom_Leg,flyto_leg,first_time_seen,number_tickets
0,GYU,YIW,2020-03-10,455
1,JJN,PKX,2020-03-12,1133
2,VDO,ICN,2020-03-12,1198
3,ICN,VDO,2020-03-12,2268
4,XUZ,KCA,2020-03-12,327


In [202]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = NewRoute_DF.first_time_seen
alert_table_DF_tmp["Date_type"] = "Issue date"
alert_table_DF_tmp["Alert"] = "Route seen for the first time"
alert_table_DF_tmp["value"] = NewRoute_DF["flyfrom_Leg"] + "-" + NewRoute_DF["flyto_leg"]  
alert_table_DF_tmp
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)

In [203]:
%%bigquery exchanges_issues_DF
SELECT * 
FROM `compliance_dashboard_IN1194.QC_exchanges_issues_refunds`
WHERE DT_OF_ISSUE > "2020-03-08"

In [204]:
exchanges_issues_DF["DT_OF_ISSUE"] = pd.to_datetime(exchanges_issues_DF['DT_OF_ISSUE'])
exchanges_issues_DF.head()

,DT_OF_ISSUE,OPERATING_CARRIER,refund,issue,exchange
0,2020-05-24,CX,0,3,0
1,2020-05-24,NZ,0,2,0
2,2020-05-24,KA,0,1,0
3,2020-03-09,BA,16949,45445,25446
4,2020-03-09,SQ,11536,16054,7948


In [205]:
too_many_refunds = exchanges_issues_DF[(exchanges_issues_DF["refund"] > exchanges_issues_DF["issue"]) & (exchanges_issues_DF["refund"] >10000)]

In [206]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = too_many_refunds.DT_OF_ISSUE
alert_table_DF_tmp["Date_type"] = "Issue date"
alert_table_DF_tmp["Alert"] = "Too many refunds for this airline"
alert_table_DF_tmp["value"] = too_many_refunds["OPERATING_CARRIER"]  
alert_table_DF_tmp
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)

In [207]:
too_many_exchanges = exchanges_issues_DF[(exchanges_issues_DF["exchange"] > exchanges_issues_DF["issue"]) & (exchanges_issues_DF["exchange"] >1000)]

In [208]:
alert_table_DF_tmp = pd.DataFrame()
alert_table_DF_tmp["Date"] = too_many_exchanges.DT_OF_ISSUE
alert_table_DF_tmp["Date_type"] = "Issue date"
alert_table_DF_tmp["Alert"] = "Too many exchanges for this airline"
alert_table_DF_tmp["value"] = too_many_exchanges["OPERATING_CARRIER"]  
alert_table_DF_tmp
alert_table_DF = alert_table_DF.append(alert_table_DF_tmp)

In [209]:
alert_table_DF

,Date,Date_type,Alert,value
42,2020-03-17,Flight Date,too many underbooked flights for the day,0.00512422
37,2020-03-18,Flight Date,too many underbooked flights for the day,0.00586853
18,2020-03-21,Flight Date,too many underbooked flights for the day,0.00905492
95,2020-03-22,Flight Date,too many underbooked flights for the day,0.00718009
24,2020-03-23,Flight Date,too many underbooked flights for the day,0.00430622
...,...,...,...,...
24817,2020-05-21,Issue date,Too many exchanges for this airline,CM
24859,2020-05-21,Issue date,Too many exchanges for this airline,SW
25083,2020-05-22,Issue date,Too many exchanges for this airline,AY
25110,2020-05-22,Issue date,Too many exchanges for this airline,DL


In [210]:
%%bigquery overbooked_flights_DF
SELECT FLIGHT_DATE, flight_id, MKT_CARRIER_CODE  
FROM `compliance_dashboard_IN1194.D_flight_capacity`
WHERE flight_capacity_max > 0 AND issued_tickets/flight_capacity_max> 3 AND issued_tickets > 100 and FLIGHT_DATE > "2020-03-08" AND FLIGHT_DATE < "2020-06-01"

In [211]:
overbooked_flights_DF["flag"] = ">300% overbooked"

In [212]:
%%bigquery nomatch_flights_DF
SELECT FLIGHT_DATE, flight_id, MKT_CARRIER_CODE  
FROM `compliance_dashboard_IN1194.D_flight_capacity`
WHERE flight_capacity_max > 0 AND  issued_tickets > 200 AND matching_rate = 0 and FLIGHT_DATE > "2020-03-08" AND FLIGHT_DATE < "2020-06-01"

In [213]:
nomatch_flights_DF["flag"] = "skytra couldn't match"

In [214]:
Suspicious_flights = overbooked_flights_DF.append(nomatch_flights_DF)
Suspicious_flights.head()

,FLIGHT_DATE,flight_id,MKT_CARRIER_CODE,flag
0,2020-05-15,ISB-LHE-00385,PK,>300% overbooked
1,2020-05-01,SKZ-KHI-00531,PK,>300% overbooked
2,2020-05-08,ISB-PEW-00660,PK,>300% overbooked
3,2020-05-20,KHI-SKZ-00532,PK,>300% overbooked
4,2020-05-07,LHE-ISB-01852,PK,>300% overbooked


In [215]:
Suspicious_flights.to_gbq(destination_table='compliance_dashboard_IN1194.Suspicious_flights',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:03,  3.20s/it]


In [216]:
alert_table_DF.to_gbq(destination_table='compliance_dashboard_IN1194.alert_table',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:02,  2.48s/it]


In [217]:
str(list(alert_table_DF[alert_table_DF.Alert == "Route seen for the first time"].value)).strip('[]')

"'GYU-YIW', 'JJN-PKX', 'VDO-ICN', 'ICN-VDO', 'XUZ-KCA', 'PKX-ZAT', 'ZAT-PKX', 'PUJ-WAW', 'DPS-WAW', 'DOY-KRY', 'CUN-WAW', 'LBV-IAD', 'ASU-FRA', 'MSY-DPS', 'MSY-CGK', 'ROB-IAD', 'LLV-NKG', 'CWJ-CSX', 'JMJ-LJG', 'CGO-XIC', 'CKG-WUA', 'KRY-DOY', 'YYA-TYN', 'JGN-PKX', 'CGO-WUA', 'RUH-RMF', 'PKX-JGN', 'CZX-JJN', 'BZX-HAK', 'BZX-XIY', 'HAK-WSK', 'WSK-XIY', 'XIY-BZX', 'HAK-BZX', 'LUM-WMT', 'JJN-YYA', 'IAD-RMF', 'DME-RMF', 'WSK-HAK', 'YYA-JJN', 'KRT-KHI', 'YTY-JJN', 'YYZ-RMF', 'CAN-XAI', 'JED-RMF', 'KGF-DEL'"

In [218]:
query = """


SELECT * FROM skytra-benchmark-rnd.matching.R6 WHERE dt_of_issue IS NOT NULL AND od_pair  IN(""" + str(list(alert_table_DF[alert_table_DF.Alert == "Route seen for the first time"].value)).strip('[]') + ")"

query

"\n\n\nSELECT * FROM skytra-benchmark-rnd.matching.R6 WHERE dt_of_issue IS NOT NULL AND od_pair  IN('GYU-YIW', 'JJN-PKX', 'VDO-ICN', 'ICN-VDO', 'XUZ-KCA', 'PKX-ZAT', 'ZAT-PKX', 'PUJ-WAW', 'DPS-WAW', 'DOY-KRY', 'CUN-WAW', 'LBV-IAD', 'ASU-FRA', 'MSY-DPS', 'MSY-CGK', 'ROB-IAD', 'LLV-NKG', 'CWJ-CSX', 'JMJ-LJG', 'CGO-XIC', 'CKG-WUA', 'KRY-DOY', 'YYA-TYN', 'JGN-PKX', 'CGO-WUA', 'RUH-RMF', 'PKX-JGN', 'CZX-JJN', 'BZX-HAK', 'BZX-XIY', 'HAK-WSK', 'WSK-XIY', 'XIY-BZX', 'HAK-BZX', 'LUM-WMT', 'JJN-YYA', 'IAD-RMF', 'DME-RMF', 'WSK-HAK', 'YYA-JJN', 'KRT-KHI', 'YTY-JJN', 'YYZ-RMF', 'CAN-XAI', 'JED-RMF', 'KGF-DEL')"

In [219]:
x = load_table(query)
x.to_gbq(destination_table='compliance_dashboard_IN1194.Tickets_from_unseen_flights',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

Reading table
Compiled the dataframe
Processed 180235 lines


1it [00:14, 14.88s/it]


In [232]:
%%bigquery kiwi_stats
SELECT * FROM `skytra-benchmark-rnd.compliance_dashboard_IN1194.D_kiwi_stat_file`

In [233]:
kiwi_stats

,src,KiwiItinAcc,AirbusItinAcc,FileCount
0,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=01,13527474991,13061634274,24.0
1,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=02,16479765967,15901820917,24.0
2,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=03,15881726341,15319689818,24.0
3,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=04,15105089513,14563466749,24.0
4,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=05,16126082963,15530736475,24.0
...,...,...,...,...
187,gs://ext-kiwi-excl-stat/stats/y=2020/m=07/d=06,4469529515,4283896556,24.0
188,gs://ext-kiwi-excl-stat/stats/y=2020/m=07/d=07,4292127234,4112286938,24.0
189,gs://ext-kiwi-excl-stat/stats/y=2020/m=07/d=08,4358654655,4176207584,24.0
190,gs://ext-kiwi-excl-stat/stats/y=2020/m=07/d=09,3983443146,3809277309,24.0


In [229]:
kiwi_stats["alert"] = np.where( abs(kiwi_stats["KiwiItinAcc"].pct_change(7)) > 0.1, "Yes", "No")
kiwi_stats["change"] = kiwi_stats["KiwiItinAcc"].pct_change(7)

In [230]:
kiwi_stats.head()

,src,KiwiItinAcc,AirbusItinAcc,FileCount,alert,change
0,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=01,13527474991,13061634274,24.0,No,NaN
1,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=02,16479765967,15901820917,24.0,No,NaN
2,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=03,15881726341,15319689818,24.0,No,NaN
3,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=04,15105089513,14563466749,24.0,No,NaN
4,gs://ext-kiwi-excl-stat/stats/y=2020/m=01/d=05,16126082963,15530736475,24.0,No,NaN


In [231]:

kiwi_stats.to_gbq(destination_table='compliance_dashboard_IN1194.kiwi_stats_alert',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:02,  2.69s/it]
